In [54]:
#add necessary libraries
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt #plotting
import pandas as pd
import numpy as np
import scipy.stats as stat
from scipy import optimize
#make sure plots are embedded into the notebook
%pylab inline 
import statsmodels.formula.api as smf
#import shapefile as shp
#from shapely.geometry.polygon import Polygon
from descartes import PolygonPatch
import os
from networkx.algorithms import community
from sklearn.mixture import GaussianMixture 

# Import required libraries
import keras
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Dropout
from keras import regularizers
from keras import optimizers
import pickle
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import set_random_seed
from keras import backend as K
import datetime
from sklearn.mixture import GaussianMixture
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
import operator

%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


/Users/hemingyi/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['f', 'matrix']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [60]:
RecordWritingPath = '/Users/hemingyi/Documents/capstone/'
TransportationDataPath = '/Users/hemingyi/Documents/capstone/transportation/'
EventDataPath = '/Users/hemingyi/Documents/capstone/event data/new/'
comboPath = '/Users/hemingyi/Documents/capstone/combo/'
# dataFile = TransportationDataPath+city+'EdgeYearwiseAggregated.csv'


In [61]:
def readTransDF(TransportationDataPath,city):
    df = pd.read_csv(TransportationDataPath+city+'EdgeDatewiseAggregated.csv')
    return df
def makeGraphfromDf(df):
    G=nx.DiGraph()
    nx.set_edge_attributes(G,'weight', 0)
    for k in df.index:
        G.add_edge(df['start_id'][k],df['end_id'][k],weight=df['amount'][k])
#     nx.write_edgelist(G, comboPath+'temp/%s.net'%city)
    return G

In [62]:
def getTimeSeries(df):
    table = pd.pivot_table(df, values='amount', index=['date'],
                    columns=['start_id','end_id'], aggfunc=np.sum, fill_value=0)
    return table

In [63]:
def anomalyDetection(y,pval = 0.2,iterN=20):
    #index of regular (non-outlier points)
    #rind=y[:,0]>-10 
    rind = np.array(range(y.shape[0]))
    
    #clustering model
    gm=GaussianMixture(n_components=3,n_init=100,max_iter=1000,random_state=0) 
    for i in range(iterN): #iterate
        print('Iteration {}'.format(i+1))  
        clustering=gm.fit(y[rind,:]) #fit EM clustering model excluding outliers
        l=clustering.score_samples(y) #estimate likelihood for each point
        Lthres=sorted(l)[int(len(l)*pval)] #anomaly threshold
        rind0=0+rind
        rind=l>Lthres #non-anomalous points
        if all(rind==rind0):
            print('Convergence in {} iterations'.format(i+1))
            break
    return l < Lthres

In [64]:
def getAutoEncoder(input_dim,encoding_dim):
    #define autoencoder architecture
    autoencoder = Sequential()
    autoencoder.add(Dense(encoding_dim[0], input_shape=(input_dim,), activation='relu'))

    for l in range(1,len(encoding_dim)):
        #autoencoder.add(Dropout(0.1))
        autoencoder.add(Dense(encoding_dim[l], input_shape=(encoding_dim[l-1],), activation='relu'))
    
    for l in range(len(encoding_dim)-1,1,-1):
        autoencoder.add(Dense(encoding_dim[l-1], input_shape=(encoding_dim[l],), activation='relu'))

    autoencoder.add(Dense(input_dim, input_shape=(encoding_dim[0],), activation='sigmoid'))

    autoencoder.summary()
    return autoencoder

In [65]:
#extract encoder layers from the networks
def encoderLayers(autoencoder,input_,n_layer):
    output_=input_
    for l in range(n_layer):
        output_=autoencoder.layers[l](output_)
    return output_

In [66]:
# import events data
def getEvents(EventDataPath,city):
    events_data =EventDataPath+city+'Events.csv'
    df_events = pd.read_csv(events_data, encoding = "ISO-8859-1", parse_dates=['Date'], infer_datetime_format=True)

    # dataframe for events
    df_finalEvents =  df_events[['Date', 'Type']]

    # list events
    lis_event = df_finalEvents['Type'].unique()
    lis_event = list(lis_event)
    return (lis_event,df_finalEvents)

In [67]:
def AutoEncoderMatrix(matrix):
    # Set random seeds
    np.random.seed(112)
    set_random_seed(22)
    input_dim = matrix.shape[1]
    encoding_dim = [500,100,15]
    autoencoder = getAutoEncoder(input_dim,encoding_dim)
    autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')
    history = autoencoder.fit(matrix, matrix,
                    epochs=80,
                    batch_size=50,
                    shuffle=True,
                    verbose=1)
    input_layer = Input(shape=(input_dim,))
    encoder = Model(input_layer, (encoderLayers(autoencoder,input_layer,len(encoding_dim))))
    reducedMatrix = encoder.predict(matrix)
    return reducedMatrix

In [69]:
f = open(RecordWritingPath+'F1ScoreAutoEncoder.txt', 'w+')
f.write('Auto Encoder\n')
for city in ['DC','Taipei','Chicago','NewYork']:
    print(city)
    f.write(city+'\n')
    data = readTransDF(TransportationDataPath,city)
    lis_event,df_finalEvents = getEvents(EventDataPath,city)
    dataTs = getTimeSeries(data)
    matrix = dataTs.values
    matrix = np.log(matrix+1)
    for i in range(matrix.shape[1]):
        matrix[:, i] = (matrix[:, i] - matrix[:, i].min()) / (matrix[:, i].max() - matrix[:, i].min())
    matrix = AutoEncoderMatrix(matrix)
    date = dataTs.index.to_frame().rename(columns={'date':'Date'})
    result = {}
    for thres in range(1,10, 1):
        th = thres/10
        print("Threshhold: ",th)
        outliers = anomalyDetection(matrix,pval = th)
        EventsDF = df_finalEvents['Date'].drop_duplicates().to_frame()
        EventsDF['Anomaly'] = True
        date['Date'] = date['Date'].astype('str')
        EventsDF['Date'] = EventsDF['Date'].astype('str')
        df = EventsDF.merge(date,on='Date',how='right')
        df['outlier'] = outliers
        Precision = len(df[(df['outlier']==True)&(df['Anomaly']==True)])/len(df[df['outlier']==True])
        Recall = len(df[(df['outlier']==True)&(df['Anomaly']==True)])/len(EventsDF)
        if Precision+Recall > 0:
            F1 = (2*Precision*Recall)/(Precision+Recall)
        else:
            F1 = 0
        result[th] = F1
    bestTh = max(result.items(), key=operator.itemgetter(1))[0]
    f.write('overall F1: '+str(result[bestTh])+', Threshold: '+str(bestTh)+'\n')
    f.write('event,single_type_recall,single_type_FPR\n')
    EventType = list(df_finalEvents['Type'].unique())
    outliers = anomalyDetection(matrix,pval = bestTh)
    for event in EventType:
        print(event)
        SingleEventDF = df_finalEvents[df_finalEvents['Type'] == event]
        SingleEventDF = SingleEventDF.drop_duplicates()
        date['Date'] = date['Date'].astype('str')
        SingleEventDF['Date'] = SingleDF['Date'].astype('str')
        SingleDF = date.merge(SingleEventDF, on='Date', how='left')
    #     SingleDF = SingleEventDF.merge(date,on='Date',how='left')
        SingleDF['outliers'] = outliers
    #     SinglePrecision = len(SingleDF[(SingleDF['outlier']==True)&(SingleDF['Type'].notnull)])/len(SingleDF[SingleDF['outlier']==True])
        SingleRecall = len(SingleDF[(SingleDF['outliers']==True)&(SingleDF['Type'].notnull())])/len(SingleEventDF)
        SingleFPR = len(SingleDF[(SingleDF['outliers']==True)&(SingleDF['Type'].isna())])/len(SingleDF[SingleDF['Type'].isna()])
        print(SingleRecall, SingleFPR)
        f.write(event+','+str(SingleRecall)+','+str(SingleFPR)+'\n')
f.close()

DC
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_71 (Dense)             (None, 500)               1020500   
_________________________________________________________________
dense_72 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_73 (Dense)             (None, 15)                1515      
_________________________________________________________________
dense_74 (Dense)             (None, 100)               1600      
_________________________________________________________________
dense_75 (Dense)             (None, 2040)              206040    
Total params: 1,279,755
Trainable params: 1,279,755
Non-trainable params: 0
_________________________________________________________________
Epoch 1/80
725/725 [==============================] - 3s 5ms/step - loss: 0.6925
Epoch 2/80
725/725 [============================

Iteration 6
Iteration 7
Iteration 8
Convergence in 8 iterations
Threshhold:  0.7
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Convergence in 10 iterations
Threshhold:  0.8
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Convergence in 9 iterations
Threshhold:  0.9
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Convergence in 6 iterations
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Convergence in 6 iterations
National Holiday
0.4878048780487805 0.39473684210526316
Protest
0.56 0.3942857142857143
Extreme Weather
0.4634146341463415 0.3961988304093567
Culture Event
0.25 0.40425531914893614
Taipei
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_76 (Dense)             (None, 500)               5832500   
_____________

638/638 [==============================] - 4s 7ms/step - loss: 0.6028
Epoch 80/80
638/638 [==============================] - 4s 7ms/step - loss: 0.6021
Threshhold:  0.1
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Convergence in 6 iterations
Threshhold:  0.2
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Convergence in 7 iterations
Threshhold:  0.3
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Convergence in 9 iterations
Threshhold:  0.4
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13
Iteration 14
Iteration 15
Iteration 16
Iteration 17
Iteration 18
Iteration 19
Iteration 20
Threshhold:  0.5
Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Convergence in 11 iterations
T

731/731 [==============================] - 1s 985us/step - loss: 0.1766
Epoch 64/80
731/731 [==============================] - 1s 794us/step - loss: 0.1766
Epoch 65/80
731/731 [==============================] - 1s 777us/step - loss: 0.1765
Epoch 66/80
731/731 [==============================] - 1s 984us/step - loss: 0.1764
Epoch 67/80
731/731 [==============================] - 1s 827us/step - loss: 0.1764
Epoch 68/80
731/731 [==============================] - 1s 805us/step - loss: 0.1764
Epoch 69/80
731/731 [==============================] - 1s 806us/step - loss: 0.1763
Epoch 70/80
731/731 [==============================] - 1s 967us/step - loss: 0.1763
Epoch 71/80
731/731 [==============================] - 1s 805us/step - loss: 0.1763
Epoch 72/80
731/731 [==============================] - 1s 808us/step - loss: 0.1762
Epoch 73/80
731/731 [==============================] - 1s 940us/step - loss: 0.1762
Epoch 74/80
731/731 [==============================] - 1s 812us/step - loss: 0.1761
Epoc

730/730 [==============================] - 34s 47ms/step - loss: 0.3756
Epoch 54/80
730/730 [==============================] - 32s 43ms/step - loss: 0.3756
Epoch 55/80
730/730 [==============================] - 30s 41ms/step - loss: 0.3755
Epoch 56/80
730/730 [==============================] - 31s 42ms/step - loss: 0.3760
Epoch 57/80
730/730 [==============================] - 27s 37ms/step - loss: 0.3759
Epoch 58/80
730/730 [==============================] - 27s 36ms/step - loss: 0.3750
Epoch 59/80
730/730 [==============================] - 27s 37ms/step - loss: 0.3751
Epoch 60/80
730/730 [==============================] - 27s 38ms/step - loss: 0.3764
Epoch 61/80
730/730 [==============================] - 27s 36ms/step - loss: 0.3753
Epoch 62/80
730/730 [==============================] - 27s 37ms/step - loss: 0.3748
Epoch 63/80
730/730 [==============================] - 27s 37ms/step - loss: 0.3750
Epoch 64/80
730/730 [==============================] - 30s 41ms/step - loss: 0.3768
Epoc